In [ ]:
import numpy as np
from lightfm.datasets import fetch_movielens
from lightfm import LightFM
from lightfm.evaluation import auc_score,recall_at_k, precision_at_k

data = fetch_movielens(min_rating =4.0) # 

#print(repr(data['train']))
#print(repr(data['test']))

model = LightFM(learning_rate=0.01,loss = 'warp')
model.fit(data['test'], epochs=35, num_threads=2)

train =data['train']
test =data['test']

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test).mean()

train_recall = recall_at_k(model, train).mean()
test_recall = recall_at_k(model, test).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))
print('Recall: train %.2f, test %.2f.' % (train_recall, test_recall))

def sample_recommendation(model, data, user_ids):
    n_users, n_items = data['train'].shape
    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]

        scores = model.predict(user_id, np.arange(n_items))
        top_items = data['item_labels']
        top_items_original = data['item_labels'][np.argsort(-scores)]

        print("User %s" % user_id)
        print("     Known positives:")
        
        for x in known_positives[:5]:
            print("        %s" % x)
        
        print("     Recommended:")
        
        for x in top_items_original[:3]:
           print("        %s" % x)
        print()
        #print(scores.shape)
        #print(top_items.shape)
        mat = np.column_stack((top_items,scores))
        mat_sort = mat[np.argsort(-scores)]
        for x in mat_sort[:5]:
            print("        %s" % x)
        print()
        #for x in mat[:3]:
        #    print("        %s" % x)


In [ ]:
sample_recommendation(model, data, [1,2,3])